# Verificando se uma série é estacionária

Para fins didáticos, vamos estudar se uma série temporal é estacionária de acordo com o Teste de Dickley-Fuller.

O conjunto de dados utilizados está disponível no site do [Portal Action](http://www.portalaction.com.br/manual-series-temporais/testes-de-estacionariedade-para-precipitacao-anual-no-lago-de-michigan). Eles utilizaram o software Action Stat, porém fica uma alternativa em python.

### Definição de série estacionária

Uma série é estacionária se as suas propriedades estatísticas se mantenham **constantes** em função do tempo. As propriedades estatísticas são:

- Média;

- Variância;

- Covariância entre os termos $T_{n}$ e $T_{n + k}$

Fazer a verificação é importante, pois muitos testes estatísticos de séries temporais supõem que elas sejam estacionárias. Caso a série não cumpra esta premissa, deve-se fazer uma transformação matemática. 

In [1]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller # importar o teste ADF

In [2]:
file = 'teste_de_estacionariedade.xlsx'
df = pd.read_excel(file)

In [3]:
df.head()

,Ano,Precipitacao
0,1900,33.20
1,1901,27.55
2,1902,31.81
3,1903,32.50
4,1904,31.01


In [4]:
df.dtypes

Ano               int64
Precipitacao    float64
dtype: object

# 1. Convertendo 'Ano' para DateTime

In [5]:
df.Ano = pd.to_datetime(df.Ano, format='%Y')

In [6]:
df.dtypes

Ano             datetime64[ns]
Precipitacao           float64
dtype: object

Uma vez convertido para `Datetime`, deixaremos a coluna `Ano` como o nosso index e, em seguida, para não conter duplicação, removeremos a coluna.

In [7]:
df.index = df.Ano
df.head()

,Ano,Precipitacao
Ano,,
1900-01-01,1900-01-01,33.20
1901-01-01,1901-01-01,27.55
1902-01-01,1902-01-01,31.81
1903-01-01,1903-01-01,32.50
1904-01-01,1904-01-01,31.01


In [9]:
df.drop('Ano', inplace = True, axis=1)

KeyError: "['Ano'] not found in axis"

In [11]:
df.dtypes

Precipitacao    float64
dtype: object

# 2. Testes estatísticos

In [12]:
# importar os dados como coluna única
df_estacionaria = pd.read_excel(file, index_col=0, squeeze=True)

In [13]:
# extrair apenas os valores
X = df_estacionaria.values

In [14]:
# aplicando o teste ADF
resultado = adfuller(X)

In [15]:
print(resultado)

(-9.81055738849395, 5.672072811950002e-17, 0, 86, {'1%': -3.5087828609430614, '5%': -2.895783561573195, '10%': -2.5850381719848565}, 388.2739863338593)


In [16]:
print('*' * 5, "RESULTADO", '*' * 5)
print("Teste Estatístico: {:.4f}".format(resultado[0]))
print("Valor-p: {:.4f}".format(resultado[1]))
print("Valores críticos: ")
for key, value in resultado[4].items():
    print("\t{}: {:.4f}".format(key, value))

***** RESULTADO *****
Teste Estatístico: -9.8106
Valor-p: 0.0000
Valores críticos: 
	1%: -3.5088
	5%: -2.8958
	10%: -2.5850


# 3. Teste de Hipótese

$H_{0}$: A série é estacionária.

$H_{1}$: A série é não-estacionária.

---

Vamos considerar os níveis de significância de $0.01 $, $0.05 $ e $ 0.1 $.

#### Nível de significância: 0.01

$Valor-p \le 0.01$: Rejeita-se $H_{0}$

$Valor-p > 0.01$: Falhamos em rejeitar $H_{0}$ e a série é **não-estacionária**.

#### Nível de significância: $0.05$

$Valor-p \le 0.05$: Rejeita-se $H_{0}$

$Valor-p > 0.05$: Falhamos em rejeitar $H_{0}$ e a série é **não-estacionária**.

#### Nível de significância: 0.10

$Valor-p \le 0.1$: Rejeita-se $H_{0}$

$Valor-p > 0.1$:Falhamos em rejeitar $H_{0}$ e a série é **não-estacionária**.

# 4. Conclusão

$H_{0}$ foi rejeitada com 1% de erro. Portanto, a série deste conjunto de dados é **não-estacionária**.